#### importing necessary libraries

In [1]:
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

##### Reading the csv file and putting in pandas data frame

In [49]:
#Read the data
df = pd.read_excel('./news.xlsx',)
#df=pd.read_csv('./news.csv')
df.columns=["title","text","type"]
#Get shape and head
df.shape
#df = df.head()
df = df.fillna(' ')
df.head()


,title,text,type
0,نقابة المهن الموسيقية تؤكد أن مهمتها ملاحقة أع...,خرجت نقابة حفظ الزمن الجميل والمهن الموسيقية ا...,مزيف
1,أسعار الأسهم بالبورصة المصرية اليوم الأحد 17-7...,"ينشر ""اليوم السابع"" أسعار أسهم أكبر 5 شركات حق...",حقيقي
2,محافظ دمياط تحذر من أى محاولات للتلاعب بتعريفة...,أكدت الدكتورة منال عوض محافظ دمياط أن المحافظ...,حقيقي
3,السيسي يأمر المواطنين بلم عواماتهم ومغادرة الن...,طلب المشير عبد الفتاح السيسي من المواطنين الوق...,مزيف
4,وزير النقل يقيل رئيس الهيئة القومية للأنفاق من...,علم اليوم السابع أن وزير النقل، أصدر قرارا بإق...,حقيقي


## Examining data set

In [50]:
df.nunique()

title    197
text     197
type       2
dtype: int64

In [51]:
df.shape

(201, 3)

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   201 non-null    object
 1   text    201 non-null    object
 2   type    201 non-null    object
dtypes: object(3)
memory usage: 4.8+ KB


In [53]:
df.axes

[RangeIndex(start=0, stop=201, step=1),
 Index(['title', 'text', 'type'], dtype='object')]

#### Importing labels

In [54]:
#Getting the labels
labels=df.type
labels.head()

0     مزيف
1    حقيقي
2    حقيقي
3     مزيف
4    حقيقي
Name: type, dtype: object

#### Split the dataset into training and testing sets

In [72]:
x_train,x_test,y_train,y_test=train_test_split(df['text'], labels, test_size=0.2, random_state=7)

#### Fitting

In [73]:
#Initialize a TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)

#Fit and transform train set, transform test set
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

In [65]:
#Initialize a PassiveAggressiveClassifier
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)
#Predict on the test set and calculate accuracy
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 97.56%


In [74]:
#Initialize a PassiveAggressiveClassifier
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)
#Predict on the test set and calculate accuracy
y_pred=pac.predict(tfidf_vectorizer.transform(df['text']))
score=accuracy_score(df['type'],y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 99.5%


In [58]:
#Build confusion matrix
confusion_matrix(y_test,y_pred, labels=['مزيف','حقيقي'])

array([[24,  0],
       [ 1, 16]], dtype=int64)

In [75]:
# function to test real or fake
def realOrFake(text):
    input = tfidf_vectorizer.transform(np.array([text]))
    output = pac.predict(input)
    return output[0]

In [63]:
validation_text = input("Put the news text here: \n") 
realOrFake(validation_text)

'حقيقي'